In [1]:
import cv2
import mediapipe as mp
import os

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Gesture detection logic
def detect_gesture(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
    thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y

    # Thumb Up Gesture
    if thumb_tip < thumb_ip and index_tip > thumb_tip and middle_tip > thumb_tip:
        return "Thumbs Up"

    # Open Palm Gesture
    if all([
        index_tip < thumb_ip,  # Fingers higher than thumb IP
        middle_tip < thumb_ip,
        ring_tip < thumb_ip,
        pinky_tip < thumb_ip
    ]):
        return "Open Palm"

    # Fist Gesture
    if all([
        thumb_tip > index_tip,
        index_tip > middle_tip,
        middle_tip > ring_tip,
        ring_tip > pinky_tip
    ]):
        return "Fist"

    return "Unknown"

def perform_action(gesture):

    if gesture == "Thumbs Up":
        print("Opening WhatsApp App...")
        os.system("start whatsapp:")  # Open WhatsApp Desktop App

    elif gesture == "Fist":
        print("Opening File Manager...")
        os.system("explorer")  # Open File Manager

    elif gesture == "Open Palm":
        print("Locking Screen...")
        os.system("rundll32.exe user32.dll, LockWorkStation")  # Lock the screen

# Capture video from the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame for natural interaction
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hand gestures
    results = hands.process(rgb_frame)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks on the frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect gesture
            gesture = detect_gesture(hand_landmarks)
            cv2.putText(frame, f"Gesture: {gesture}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            # Perform action for the detected gesture
            if gesture != "Unknown":
                perform_action(gesture)

    # Display the video frame
    cv2.imshow("Gesture Control", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

Opening WhatsApp App...
Opening WhatsApp App...
Opening WhatsApp App...
